In [1]:
from Pointcloud.Modules.Alignor import Alignor
from Pointcloud.Modules import Config as config
from Pointcloud.Modules.FileDataset import FileDataset
from Pointcloud.Modules.GraphBuilder import GraphBuilder
from Pointcloud.Modules.Manager import Manager
from Pointcloud.Modules.Model import Patch2NormalModel
from Pointcloud.Modules.Noise import Noise
from Pointcloud.Modules.Object import FilePointcloud, FileMesh, Pointcloud
from Pointcloud.Modules.Preprocessor import Preprocessor
from Pointcloud.Modules.FileDataset import FileDataset, NoiseLevels
from Pointcloud.Modules.Selector import Selector
from Pointcloud.Modules.Utils import GeneralUtils, TorchUtils, SlicedTorchData
from Pointcloud.Modules.Visualize import visualize_coloring, plot_2d

from copy import deepcopy
from deltaconv.geometry import estimate_basis
from igl import (
    avg_edge_length as igl_avg_edge_length,
    barycenter as igl_barycenter,
    bounding_box_diagonal as igl_bounding_box_diagonal,
    per_vertex_normals as igl_per_vertex_normals,
    read_obj as igl_read_obj,
    triangle_triangle_adjacency as igl_triangle_triangle_adjacency,
    write_obj as igl_write_obj
)
from itertools import zip_longest
from meshplot import plot as mp_plot
from gravomg.util import neighbors_from_stiffness
import numpy as np
from numpy import random as np_random
from pathlib import Path
import pytorch_lightning as pl
import matplotlib.pyplot as plt
from robust_laplacian import point_cloud_laplacian
from scipy.spatial import Delaunay,\
    KDTree
from sklearn.preprocessing import normalize as sklearn_preprocessing_normalize
from timeit import timeit
import torch_geometric as tg
from torch_cluster import knn_graph
import torch
from tqdm import tqdm

EXAMPLE_OBJ_FILE = "common-3d-test-models-master/stanford-bunny.obj"
# EXAMPLE_OBJ_FILE = "common-3d-test-models-master/fandisk.obj"
# EXAMPLE_OBJ_FILE = "common-3d-test-models-master/beetle.obj"
# EXAMPLE_OBJ_FILE = "untitled.obj"

c:\Users\ruben\miniconda3\envs\PCDenoiser\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
c:\Users\ruben\miniconda3\envs\PCDenoiser\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
manager = Manager()
pointcloud = Pointcloud.loadObj(EXAMPLE_OBJ_FILE)
pp = Preprocessor(pointcloud)
graph = pp.graph
noise = Noise(graph)
noise.generateNoise(0.1, 0, 0)
# prediction = manager.predict(pp, "tb_logs/testmodel/version_22/checkpoints/0-epoch50-00.62.ckpt")

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
pointcloud2 = Pointcloud.loadObj(EXAMPLE_OBJ_FILE)
pp2 = Preprocessor(pointcloud2)
gt = pp2.graph.n
_scale = 200
start = graph.pos.numpy()
endpred = start + prediction.numpy() / _scale
endgt = start + gt.numpy() / _scale
endinput = start + graph.n.numpy() / _scale
plot = mp_plot(start)
plot.add_lines(start, endpred, shading={"line_color": "green"})
plot.add_lines(start, endgt, shading={"line_color": "yellow"})
plot.add_lines(start, endinput, shading={"line_color": "purple"})

['file_path', 'hasFilePath', 'hasNormals', 'loadObj', 'n', 'saveObj', 'v']


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016867…

3